In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
import cv2
import h5py
from PIL import Image
import torch
import torch.nn as nn
from transformers import ViTModel, ViTImageProcessor
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score, f1_score
import time
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
import os


In [2]:
class ISICDataset(Dataset):
    def __init__(self, df, hdf5_file, processor, model_type="vit", neg_sample=0.01, pos_sample=5.0, seed=42):
        self.hdf5_file = hdf5_file
        self.processor = processor
        self.model_type = model_type
        self.seed = seed

        # Apply balanced sampling
        self.df = self._balance_sampling(df, neg_sample, pos_sample)

    def _balance_sampling(self, df, neg_sample, pos_sample):
        positive_df = df.query("target == 0").sample(frac=neg_sample, random_state=self.seed)
        negative_df = df.query("target == 1").sample(frac=pos_sample, replace=True, random_state=self.seed)
        balanced_df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0, random_state=self.seed)
        return balanced_df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        isic_id = self.df.iloc[idx]['isic_id']
        label = torch.tensor(self.df.iloc[idx]['target'], dtype=torch.float32)

        image_data = self.hdf5_file[isic_id][()]
        image_array = np.frombuffer(image_data, np.uint8)
        image = cv2.cvtColor(cv2.imdecode(image_array, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

        if self.model_type == "vit":
            # For ViT, use the processor directly
            inputs = self.processor(images=image, return_tensors="pt")
            image = inputs['pixel_values'].squeeze()
        else:
            # For EfficientNet, convert numpy array to PIL image and apply transforms
              # Convert to PIL format
            image = self.processor(Image.fromarray(image))   # Apply EfficientNet transformations

        return image, label

In [3]:
class ISICModel(nn.Module):
    def __init__(self, model_name="vit"):
        super(ISICModel, self).__init__()
        self.model_name = model_name
        
        if model_name == "vit":
            self.model = ViTModel.from_pretrained('google/vit-base-patch16-224')
            self.classifier = nn.Linear(self.model.config.hidden_size, 1)
        elif model_name == "effnet_b0":
            self.model = models.efficientnet_b0(pretrained=True)
            self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, 1)
            self.classifier = self.model.classifier[1]
        elif model_name == "effnet_b4":
            self.model = models.efficientnet_b4(pretrained=True)
            self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, 1)
            self.classifier = self.model.classifier[1]
        else:
            raise ValueError("Model name must be 'vit', 'effnet_b0', or 'effnet_b4'")
            
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if self.model_name == "vit":
            outputs = self.model(pixel_values=x)
            x = outputs.last_hidden_state[:, 0, :]  # CLS token
            x = self.classifier(x)
        else:
            x = self.model(x)
        return self.sigmoid(x).squeeze()

In [4]:
# Function to calculate accuracy
def calculate_accuracy(y_true, y_pred):
    y_pred = torch.round(y_pred)  # Convert probabilities to binary labels
    return (y_true == y_pred).float().mean()

# Custom F1-Score calculation for binary classification
def calculate_f1_torch(true_labels, predicted_labels):
    tp = (true_labels * predicted_labels).sum().float()
    fp = ((1 - true_labels) * predicted_labels).sum().float()
    fn = (true_labels * (1 - predicted_labels)).sum().float()
    
    precision = tp / (tp + fp + 1e-8)  # Adding epsilon to avoid division by zero
    recall = tp / (tp + fn + 1e-8)
    
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1.item()

# Custom AUC-ROC calculation for binary classification
def roc_auc_torch(y_true, y_scores):
    # Sort scores and corresponding true labels
    desc_score_indices = torch.argsort(y_scores, descending=True)
    y_scores_sorted = y_scores[desc_score_indices]
    y_true_sorted = y_true[desc_score_indices]
    
    # Compute true positive and false positive rates
    tps = torch.cumsum(y_true_sorted, dim=0)
    fps = torch.cumsum(1 - y_true_sorted, dim=0)

    # Add a 0 at the beginning for FPR and TPR
    tps = torch.cat([torch.tensor([0.0], device=tps.device), tps])
    fps = torch.cat([torch.tensor([0.0], device=fps.device), fps])

    # Normalize tps and fps to get the TPR and FPR (False Positive Rate)
    tpr = tps / tps[-1]  # True positive rate (recall)
    fpr = fps / fps[-1]  # False positive rate

    # Calculate AUC as the area under the TPR-FPR curve (trapezoidal rule)
    auc = torch.trapz(tpr, fpr)  # Use torch.trapz to compute the integral
    return auc.item()

In [5]:
class ModelTrainer:
    def __init__(self, model, train_loader, val_loader, device="cuda", log_interval=100):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.log_interval = log_interval

    def train(self, epochs=5):
        for epoch in range(epochs):
            epoch_start_time = time.time()
            self.model.train()
            train_loss, train_acc, train_f1 = 0.0, 0.0, 0.0
            true_labels, prob_preds = [], []

            # Training loop with step-wise output
            for step, (images, labels) in enumerate(self.train_loader, 1):
                images, labels = images.to(self.device), labels.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs.squeeze(), labels)
                loss.backward()
                self.optimizer.step()

                # Collect metrics for each step
                train_loss += loss.item()
                accuracy = (torch.round(outputs.squeeze()) == labels).float().mean().item()
                f1 = calculate_f1_torch(labels, torch.round(outputs).squeeze())  # Custom F1 function
                train_acc += accuracy
                train_f1 += f1
                true_labels.extend(labels.cpu().numpy())
                prob_preds.extend(outputs.detach().cpu().numpy())

                # Log at specified intervals
                if step % self.log_interval == 0:
                    avg_loss = train_loss / step
                    avg_acc = train_acc / step
                    avg_f1 = train_f1 / step
                    print(f'Epoch [{epoch+1}/{epochs}], Step [{step}/{len(self.train_loader)}], '
                          f'Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}, F1-Score: {avg_f1:.4f}')

            # Calculate epoch metrics
            epoch_loss = train_loss / len(self.train_loader)
            epoch_acc = train_acc / len(self.train_loader)
            epoch_f1 = train_f1 / len(self.train_loader)
            epoch_auc = roc_auc_torch(torch.tensor(true_labels), torch.tensor(prob_preds))  # Custom AUC function
            epoch_time = (time.time() - epoch_start_time) / 60

            print(f'End of Epoch [{epoch+1}/{epochs}] -> '
                  f'Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, F1-Score: {epoch_f1:.4f}, '
                  f'AUC-ROC: {epoch_auc:.4f}, Time: {epoch_time:.2f} mins')

            # Validation step at the end of each epoch
            self._validate()

    def _validate(self):
        self.model.eval()  # Set model to evaluation mode
        val_loss, val_acc, val_f1 = 0.0, 0.0, 0.0
        true_labels, prob_preds = [], []

        with torch.no_grad():
            for images, labels in self.val_loader:
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs.squeeze(), labels)

                val_loss += loss.item()
                accuracy = (torch.round(outputs.squeeze()) == labels).float().mean().item()
                f1 = calculate_f1_torch(labels, torch.round(outputs).squeeze())  # Custom F1 function

                val_acc += accuracy
                val_f1 += f1
                true_labels.extend(labels.cpu().numpy())
                prob_preds.extend(outputs.cpu().numpy())

        # Average metrics for validation
        val_loss /= len(self.val_loader)
        val_acc /= len(self.val_loader)
        val_f1 /= len(self.val_loader)
        val_auc = roc_auc_torch(torch.tensor(true_labels), torch.tensor(prob_preds))  # Custom AUC function

        print(f'Validation -> Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}, F1-Score: {val_f1:.4f}, AUC-ROC: {val_auc:.4f}')
        
        def predict(self, dataloader):
            """Method to predict outputs on a new dataset"""
            self.model.eval()
            predictions = []

            with torch.no_grad():
                for images, _ in dataloader:
                    images = images.to(self.device)
                    outputs = self.model(images)
                    predictions.extend(outputs.squeeze().cpu().numpy())

            return predictions

In [ ]:
# Configuration and data paths
# Options: 'vit', 'effnet_b0', 'effnet_b4'
train_hdf5_path = '/kaggle/input/isic-2024-challenge/train-image.hdf5'
train_metadata_path = '/kaggle/input/isic-2024-challenge/train-metadata.csv'
train_hdf5 = h5py.File(train_hdf5_path, 'r')
train_metadata = pd.read_csv(train_metadata_path)
# Initialize datasets and dataloaders as before


# Load metadata and perform train-validation split
train_metadata = pd.read_csv(train_metadata_path)
train_df_isic, val_df_isic = train_test_split(train_metadata, test_size=0.2, random_state=42)

In [11]:
def trainig_isic(model_name, epochs=5):    
    # Dataset transformations based on model
    if model_name == "vit":
        processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
        transform = None  # Processor will handle transformations
    else:
        processor = transforms.Compose([
            transforms.Resize(224 if model_name == "effnet_b0" else 380),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])


    # Initialize the datasets with the split data
    train_dataset = ISICDataset(train_df_isic, train_hdf5, processor, model_type=model_name)
    val_dataset = ISICDataset(val_df_isic, train_hdf5, processor, model_type=model_name)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

    # Model and Trainer with validation
    model = ISICModel(model_name=model_name)

    trainer = ModelTrainer(model, train_loader, val_loader, log_interval=100)
    trainer.train(epochs)
    
    return model



In [ ]:
isic_model_vit = trainig_isic(model_name = "vit")

In [7]:
isic_model_effnet_b4 = trainig_isic(model_name = "effnet_b4")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:01<00:00, 57.6MB/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded 

Epoch [1/5], Step [100/300], Loss: 0.6378, Accuracy: 0.6400, F1-Score: 0.0478
Epoch [1/5], Step [200/300], Loss: 0.5721, Accuracy: 0.6916, F1-Score: 0.2010


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch [1/5], Step [300/300], Loss: 0.5084, Accuracy: 0.7396, F1-Score: 0.3729
End of Epoch [1/5] -> Loss: 0.5084, Accuracy: 0.7396, F1-Score: 0.3729, AUC-ROC: 0.7925, Time: 1.96 mins
Validation -> Loss: 0.3686, Accuracy: 0.8345, F1-Score: 0.6759, AUC-ROC: 0.9037
Epoch [2/5], Step [100/300], Loss: 0.2909, Accuracy: 0.8756, F1-Score: 0.7995
Epoch [2/5], Step [200/300], Loss: 0.2589, Accuracy: 0.8916, F1-Score: 0.8252
Epoch [2/5], Step [300/300], Loss: 0.2424, Accuracy: 0.9004, F1-Score: 0.8385
End of Epoch [2/5] -> Loss: 0.2424, Accuracy: 0.9004, F1-Score: 0.8385, AUC-ROC: 0.9597, Time: 1.95 mins
Validation -> Loss: 0.5545, Accuracy: 0.8229, F1-Score: 0.6467, AUC-ROC: 0.8813
Epoch [3/5], Step [100/300], Loss: 0.1410, Accuracy: 0.9456, F1-Score: 0.9056
Epoch [3/5], Step [200/300], Loss: 0.1278, Accuracy: 0.9525, F1-Score: 0.9211
Epoch [3/5], Step [300/300], Loss: 0.1234, Accuracy: 0.9531, F1-Score: 0.9191
End of Epoch [3/5] -> Loss: 0.1234, Accuracy: 0.9531, F1-Score: 0.9191, AUC-ROC: 0.9

In [8]:
isic_model_effnet_b0 = trainig_isic(model_name = "effnet_b0")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 70.7MB/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded 

Epoch [1/5], Step [100/300], Loss: 0.5010, Accuracy: 0.7806, F1-Score: 0.6888
Epoch [1/5], Step [200/300], Loss: 0.4128, Accuracy: 0.8228, F1-Score: 0.7348


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch [1/5], Step [300/300], Loss: 0.3585, Accuracy: 0.8494, F1-Score: 0.7703
End of Epoch [1/5] -> Loss: 0.3585, Accuracy: 0.8494, F1-Score: 0.7703, AUC-ROC: 0.9176, Time: 0.34 mins
Validation -> Loss: 0.4086, Accuracy: 0.8297, F1-Score: 0.6749, AUC-ROC: 0.9008
Epoch [2/5], Step [100/300], Loss: 0.1554, Accuracy: 0.9513, F1-Score: 0.9084
Epoch [2/5], Step [200/300], Loss: 0.1437, Accuracy: 0.9509, F1-Score: 0.9145
Epoch [2/5], Step [300/300], Loss: 0.1374, Accuracy: 0.9510, F1-Score: 0.9170
End of Epoch [2/5] -> Loss: 0.1374, Accuracy: 0.9510, F1-Score: 0.9170, AUC-ROC: 0.9876, Time: 0.34 mins
Validation -> Loss: 0.5362, Accuracy: 0.8212, F1-Score: 0.6418, AUC-ROC: 0.8904
Epoch [3/5], Step [100/300], Loss: 0.0735, Accuracy: 0.9794, F1-Score: 0.9669
Epoch [3/5], Step [200/300], Loss: 0.0764, Accuracy: 0.9756, F1-Score: 0.9595
Epoch [3/5], Step [300/300], Loss: 0.0736, Accuracy: 0.9756, F1-Score: 0.9562
End of Epoch [3/5] -> Loss: 0.0736, Accuracy: 0.9756, F1-Score: 0.9562, AUC-ROC: 0.9

In [9]:
# Define paths to save each model's weights in Kaggle's working directory
vit_path = "/kaggle/working/isic_model_weights_vit.pth"
effnet_b0_path = "/kaggle/working/isic_model_weights_effnet_b0.pth"
effnet_b4_path = "/kaggle/working/isic_model_weights_effnet_b4.pth"

# Save each model's state_dict
torch.save(isic_model_vit.state_dict(), vit_path)
print("Model weights saved successfully in Kaggle.")

Model weights saved successfully in Kaggle.


In [ ]:
torch.save(isic_model_effnet_b0.state_dict(), effnet_b0_path)
torch.save(isic_model_effnet_b4.state_dict(), effnet_b4_path)

print("Model weights saved successfully in Kaggle.")

In [6]:
df = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')  # Uncomment and adjust if loading from a file

# Dictionary to map lesion types to benign or malignant
benign_types = ['nv', 'bkl', 'akiec', 'vasc', 'df']
malignant_types = ['mel', 'bcc']

# Add binary target column: 0 for benign, 1 for malignant
df['target'] = df['dx'].apply(lambda x: 0 if x in benign_types else 1)

In [7]:
# Define the folder paths
folder_1 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1"
folder_2 = "/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2"

# Function to locate the image path
def get_image_path(image_id):
    filename = f"{image_id}.jpg"  # Assuming images are named after image_id with .jpg extension
    if os.path.exists(os.path.join(folder_1, filename)):
        return os.path.join(folder_1, filename)
    elif os.path.exists(os.path.join(folder_2, filename)):
        return os.path.join(folder_2, filename)
    else:
        return None  # In case the image is missing

    
df['image_path'] = df['image_id'].apply(get_image_path)

In [8]:
class HAM10000Dataset(Dataset):
    def __init__(self, df, processor, model_type="vit", seed=42):
        """
        Args:
            df (DataFrame): DataFrame containing the dataset metadata.
            processor: Preprocessing function or model-specific transform.
            model_type (str): Model type ("vit" or "efficientnet") to determine preprocessing.
            seed (int): Seed for reproducibility.
        """
        self.processor = processor
        self.model_type = model_type
        self.seed = seed
        self.df = df


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        label = torch.tensor(row['target'], dtype=torch.float32)

        # Load and process the image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.model_type == "vit":
            # Process for ViT model
            inputs = self.processor(images=image, return_tensors="pt")
            image = inputs['pixel_values'].squeeze()  # Shape: [3, height, width]
        else:
            # Process for EfficientNet model
            image = self.processor(Image.fromarray(image))  # Apply EfficientNet transformations

        return image, label


In [9]:
train_df_ham10000, val_df_ham10000 = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
def train_ham10000(model_name, epochs=5):
    # Automatically set up preprocessing based on EfficientNet model size
    def get_efficientnet_preprocessor(model_name):
        target_size = 224 if model_name == "effnet_b0" else 380
        return transforms.Compose([
            transforms.Resize(target_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    # Dataset transformations based on model
    if model_name == "vit":
        processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
        transform = None  # Processor will handle transformations
    else:
        processor = get_efficientnet_preprocessor(model_name)

    train_dataset = HAM10000Dataset(train_df_ham10000, processor, model_type=model_name)
    val_dataset = HAM10000Dataset(val_df_ham10000, processor, model_type=model_name)


    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)

    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

    model = ISICModel(model_name=model_name)

    trainer = ModelTrainer(model, train_loader, val_loader, log_interval=100)
    trainer.train(epochs)
    
    return model
    


In [11]:
ham_model_vit = train_ham10000("vit", epochs=2)

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch [1/2], Step [100/501], Loss: 0.3742, Accuracy: 0.8363, F1-Score: 0.2074
Epoch [1/2], Step [200/501], Loss: 0.3446, Accuracy: 0.8462, F1-Score: 0.2655
Epoch [1/2], Step [300/501], Loss: 0.3363, Accuracy: 0.8504, F1-Score: 0.3044
Epoch [1/2], Step [400/501], Loss: 0.3241, Accuracy: 0.8547, F1-Score: 0.3152
Epoch [1/2], Step [500/501], Loss: 0.3187, Accuracy: 0.8561, F1-Score: 0.3394


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


End of Epoch [1/2] -> Loss: 0.3186, Accuracy: 0.8562, F1-Score: 0.3401, AUC-ROC: 0.8571, Time: 2.58 mins
Validation -> Loss: 0.2711, Accuracy: 0.8920, F1-Score: 0.4697, AUC-ROC: 0.9081
Epoch [2/2], Step [100/501], Loss: 0.2314, Accuracy: 0.9106, F1-Score: 0.5747
Epoch [2/2], Step [200/501], Loss: 0.2464, Accuracy: 0.8969, F1-Score: 0.5501
Epoch [2/2], Step [300/501], Loss: 0.2328, Accuracy: 0.9021, F1-Score: 0.5818
Epoch [2/2], Step [400/501], Loss: 0.2392, Accuracy: 0.8978, F1-Score: 0.5488
Epoch [2/2], Step [500/501], Loss: 0.2374, Accuracy: 0.8990, F1-Score: 0.5520
End of Epoch [2/2] -> Loss: 0.2372, Accuracy: 0.8990, F1-Score: 0.5522, AUC-ROC: 0.9277, Time: 2.58 mins
Validation -> Loss: 0.2583, Accuracy: 0.8929, F1-Score: 0.5299, AUC-ROC: 0.9095


In [15]:
ham_model_effnet_b0 = train_ham10000(model_name = "effnet_b0")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/5], Step [100/501], Loss: 0.4433, Accuracy: 0.8244, F1-Score: 0.1369
Epoch [1/5], Step [200/501], Loss: 0.3897, Accuracy: 0.8359, F1-Score: 0.2068
Epoch [1/5], Step [300/501], Loss: 0.3623, Accuracy: 0.8448, F1-Score: 0.2910
Epoch [1/5], Step [400/501], Loss: 0.3481, Accuracy: 0.8497, F1-Score: 0.3332
Epoch [1/5], Step [500/501], Loss: 0.3379, Accuracy: 0.8550, F1-Score: 0.3659
End of Epoch [1/5] -> Loss: 0.3374, Accuracy: 0.8553, F1-Score: 0.3651, AUC-ROC: 0.8380, Time: 0.85 mins
Validation -> Loss: 0.2641, Accuracy: 0.8877, F1-Score: 0.4733, AUC-ROC: 0.9158
Epoch [2/5], Step [100/501], Loss: 0.2346, Accuracy: 0.8906, F1-Score: 0.5193
Epoch [2/5], Step [200/501], Loss: 0.2377, Accuracy: 0.8925, F1-Score: 0.5695
Epoch [2/5], Step [300/501], Loss: 0.2334, Accuracy: 0.8954, F1-Score: 0.5766
Epoch [2/5], Step [400/501], Loss: 0.2352, Accuracy: 0.8953, F1-Score: 0.5828
Epoch [2/5], Step [500/501], Loss: 0.2327, Accuracy: 0.8979, F1-Score: 0.5975
End of Epoch [2/5] -> Loss: 0.2325,

In [11]:
ham_model_effnet_b4 = train_ham10000(model_name = "effnet_b4")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 206MB/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded c

Epoch [1/5], Step [100/501], Loss: 0.4827, Accuracy: 0.8175, F1-Score: 0.0206
Epoch [1/5], Step [200/501], Loss: 0.4157, Accuracy: 0.8319, F1-Score: 0.0103
Epoch [1/5], Step [300/501], Loss: 0.3914, Accuracy: 0.8337, F1-Score: 0.0465
Epoch [1/5], Step [400/501], Loss: 0.3748, Accuracy: 0.8378, F1-Score: 0.0978
Epoch [1/5], Step [500/501], Loss: 0.3604, Accuracy: 0.8421, F1-Score: 0.1200


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


End of Epoch [1/5] -> Loss: 0.3603, Accuracy: 0.8421, F1-Score: 0.1197, AUC-ROC: 0.7970, Time: 4.28 mins
Validation -> Loss: 0.2977, Accuracy: 0.8702, F1-Score: 0.3891, AUC-ROC: 0.8867
Epoch [2/5], Step [100/501], Loss: 0.2729, Accuracy: 0.8788, F1-Score: 0.3930
Epoch [2/5], Step [200/501], Loss: 0.2801, Accuracy: 0.8747, F1-Score: 0.4215
Epoch [2/5], Step [300/501], Loss: 0.2771, Accuracy: 0.8742, F1-Score: 0.4103
Epoch [2/5], Step [400/501], Loss: 0.2727, Accuracy: 0.8764, F1-Score: 0.4480
Epoch [2/5], Step [500/501], Loss: 0.2690, Accuracy: 0.8785, F1-Score: 0.4657
End of Epoch [2/5] -> Loss: 0.2687, Accuracy: 0.8787, F1-Score: 0.4668, AUC-ROC: 0.9069, Time: 4.25 mins
Validation -> Loss: 0.2568, Accuracy: 0.8841, F1-Score: 0.4457, AUC-ROC: 0.9187
Epoch [3/5], Step [100/501], Loss: 0.2045, Accuracy: 0.9187, F1-Score: 0.6465
Epoch [3/5], Step [200/501], Loss: 0.1994, Accuracy: 0.9178, F1-Score: 0.6481
Epoch [3/5], Step [300/501], Loss: 0.2038, Accuracy: 0.9146, F1-Score: 0.6309
Epoch 

In [12]:
# Define paths to save each model's weights in Kaggle's working directory
vit_path = "/kaggle/working/ham_model_weights_vit.pth"
effnet_b0_path = "/kaggle/working/ham_model_weights_effnet_b0.pth"
effnet_b4_path = "/kaggle/working/ham_model_weights_effnet_b4.pth"

# Save each model's state_dict
torch.save(ham_model_vit.state_dict(), vit_path)
print("Saved ViT")

Saved ViT


In [14]:
import os
import zipfile
from datetime import datetime

def zip_model_weights(model_path='ham_model_weights_vit.pth', output_dir='/kaggle/working/'):
    # Create timestamp for unique filename
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = f'model_weights_{timestamp}.zip'
    zip_filepath = os.path.join(output_dir, zip_filename)
    
    # Create zip file
    with zipfile.ZipFile(zip_filepath, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add the model weights file to the zip
        zipf.write(model_path, os.path.basename(model_path))
    
    print(f'Model weights have been zipped to: {zip_filepath}')
    return zip_filepath

# Usage
zip_path = zip_model_weights()
print(f'You can find your zipped file at: {zip_path}')

Model weights have been zipped to: /kaggle/working/model_weights_20241113_085518.zip
You can find your zipped file at: /kaggle/working/model_weights_20241113_085518.zip


In [17]:
# If you just want to download the .pth file directly without zipping
from IPython.display import FileLink
FileLink('ham_model_weights_vit.pth')

/kaggle/working/ham_model_weights_vit.pth

In [ ]:
torch.save(ham_model_effnet_b0.state_dict(), effnet_b0_path)
torch.save(ham_model_effnet_b4.state_dict(), effnet_b4_path)